In [1]:
import pandas as pd
import re
import collections
import nltk
nltk.download()
from textblob import TextBlob
import os

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [4]:
data_file_path = os.path.abspath(os.path.join(os.pardir,'data','real_time_data_processed.csv'))
data = pd.read_csv(data_file_path)
data = data.drop('Unnamed: 0', axis=1)
data.head()

,News Category,Title,Description
0,sports,"['free', 'agent', 'damontae', 'kazee', 'visit'...","['year', 'salary', 'cap', 'mismanagement', 'ne..."
1,sports,"['desean', 'jackson', 'agrees', 'deal', 'ram']","['desean', 'jackson', 'will', 'always', 'apart..."
2,sports,"['rocket', 'coach', 'stephen', 'silas', 'heart...","['houston', 'rocket', 'case', 'havent', 'notic..."
3,sports,"['rice', 'choreographs', 'murray', 'st', 'past...","['preston', 'rice', 'threw', 'pair', 'touchdow..."
4,sports,"['celtic', 'trade', 'rumor', 'aaron', 'gordon'...","['celtic', 'got', 'perhaps', 'important', 'ext..."


# Combine Title and Description

In [5]:
data['Documents'] = data['Title'] + data['Description']
data.drop(['Title','Description'],axis=1,inplace=True)
data['Documents'] = data['Documents'].apply(lambda x: x.replace("\'","").replace(',','').replace(']','').replace('[',''))
data['Word Count'] = data['Documents'].apply(lambda x: len(re.findall(r'\w+', x)))

In [6]:
data.head()

,News Category,Documents,Word Count
0,sports,free agent damontae kazee visit detroit lionye...,29
1,sports,desean jackson agrees deal ramdesean jackson w...,25
2,sports,rocket coach stephen silas heartbreaking press...,39
3,sports,rice choreographs murray st past tennessee sta...,38
4,sports,celtic trade rumor aaron gordon sought multipl...,37


# Create Features 
### count: noun phrase, noun, verb, adjective

In [7]:
data['Noun Phrases'] = data['Documents'].apply(lambda x: len(TextBlob(x).noun_phrases))

In [8]:
data['Tags'] = data['Documents'].apply(lambda t: collections.Counter(tag for word,tag in TextBlob(t).tags))

In [9]:
# pos tag list https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

data['Noun Count'] = data['Tags'].apply(lambda d: d.get('NN',0)+d.get('NNS',0)+d.get('NNP',0)+d.get('NNPS',0))
data['Adjective Count'] = data['Tags'].apply(lambda d: d.get('JJ',0)+d.get('JJR',0)+d.get('JJS',0))
data['Verb Count'] = data['Tags'].apply(lambda d: d.get('VB',0)+d.get('VBD',0)+d.get('VBG',0)+d.get('VBN',0)+d.get('VBP',0)+d.get('VBZ',0))
data.drop(['Tags'],axis=1,inplace=True)

# Export Cleaned Data as CSV

In [25]:
data.head()

,News Category,Documents,Word Count,Noun Phrases,Noun Count,Adjective Count,Verb Count
0,sports,free agent damontae kazee visit detroit lionye...,29,4,15,8,5
1,sports,desean jackson agrees deal ramdesean jackson w...,25,6,10,6,5
2,sports,rocket coach stephen silas heartbreaking press...,39,6,19,5,8
3,sports,rice choreographs murray st past tennessee sta...,38,7,20,5,7
4,sports,celtic trade rumor aaron gordon sought multipl...,37,5,18,6,6


In [11]:
export_file_path = os.path.abspath(os.path.join(os.pardir,'data','cleaned_bing.csv'))
data.to_csv(export_file_path,index=False)